# Machine Learning Project

## Introduction

The aim of this project is to predict the presence of heart disease using a dataset obtained from the UCL Machine Learning repository. The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/statlog+%28heart%29). 

*INSERT OUTLINE OF METHODOLOGY AFTER PROJECT*

## Data Description

Our data consists of 270 instances with 13 descriptive features, and a target feature having two classes which indicate the presence or absence of heart disease. Hence, this is a binary classification problem. As is, our dataset contains no missing values.

The following explanation of the descriptive features is extracted from the dataset description on the [UCL website](https://archive.ics.uci.edu/ml/datasets/statlog+%28heart%29):
* age - **numerical**
* sex - **binary**
* chest pain type - **nominal**
* resting blood pressure - **numerical**
* serum cholestoral in mg/dl - **numerical**      
* fasting blood sugar > 120 mg/dl - **binary**
* resting electrocardiographic results - **nominal**
* maximum heart rate achieved - **numerical**
* exercise induced angina - **binary**
* oldpeak = ST depression induced by exercise relative to rest - **numerical**
* the slope of the peak exercise ST segment - **ordinal**
* number of major vessels (0-3) colored by flourosopy - **numerical**    
* thal - **nominal**

Each of the nominal features has been integer encoded, which is a miselading representation of the data and will be addressed in the processing stage. 

Time to import our dataset as well as any modules we will be using, and do some preliminary configuration.

In [15]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

# Ignore python warnings
import warnings
warnings.filterwarnings('ignore')

# display all columns
pd.set_option('display.max_columns', None) 

# read in and configure data with column names
# col_names = ['age', 'sex', 'chest_pain_type', 'resting_bp', 'serum_cholestoral', 'fasting_blood_sugar', 'resting_ecg_results', 'max_hr_achieved', 'exercise_induced_angina', 'oldpeak', 'slope_of_peak_exercise', 'no_of_major_vessels', 'thal', 'target']
# these names are abbreviated below for space
col_names = ['age', 'sex', 'cpt', 'rb', 'sc', 'fbs', 'rer', 'mha', 'eia', 'old', 'sope', 'nomv', 'thal', 'target'] 
data = pd.read_csv('heart.csv',names=col_names,header=None)

# separate target feature
target = data['target']
data.drop(columns=['target'],inplace=True)

Now that we have our dataset imported and our target feature separated we can have a brief look at our dataset using python.

In [24]:
print("----------")
print(f"Shape of the dataset is {data.shape} \n")
print("----------")
print(f"Each of the descriptive features have the following types:\n{data.dtypes}\n")
print("----------")
print(f"Each of the descriptive features have the following number of unique values:\n{data.nunique()}\n")
print("----------")
print(f"The dataset contains no missing values:\n{data.isna().sum()}\n")
data.head(20)

----------
Shape of the dataset is (270, 13) 

----------
Each of the descriptive features have the following types:
age     float64
sex     float64
cpt     float64
rb      float64
sc      float64
fbs     float64
rer     float64
mha     float64
eia     float64
old     float64
sope    float64
nomv    float64
thal    float64
dtype: object

----------
Each of the descriptive features have the following number of unique values:
age      41
sex       2
cpt       4
rb       47
sc      144
fbs       2
rer       3
mha      90
eia       2
old      39
sope      3
nomv      4
thal      3
dtype: int64

----------
The dataset contains no missing values:
age     0
sex     0
cpt     0
rb      0
sc      0
fbs     0
rer     0
mha     0
eia     0
old     0
sope    0
nomv    0
thal    0
dtype: int64



,age,sex,cpt,rb,sc,fbs,rer,mha,eia,old,sope,nomv,thal
0,70.0,1.0,4.0,130.0,322.0,0.0,2.0,109.0,0.0,2.4,2.0,3.0,3.0
1,67.0,0.0,3.0,115.0,564.0,0.0,2.0,160.0,0.0,1.6,2.0,0.0,7.0
2,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0
3,64.0,1.0,4.0,128.0,263.0,0.0,0.0,105.0,1.0,0.2,2.0,1.0,7.0
4,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0
5,65.0,1.0,4.0,120.0,177.0,0.0,0.0,140.0,0.0,0.4,1.0,0.0,7.0
6,56.0,1.0,3.0,130.0,256.0,1.0,2.0,142.0,1.0,0.6,2.0,1.0,6.0
7,59.0,1.0,4.0,110.0,239.0,0.0,2.0,142.0,1.0,1.2,2.0,1.0,7.0
8,60.0,1.0,4.0,140.0,293.0,0.0,2.0,170.0,0.0,1.2,2.0,2.0,7.0
9,63.0,0.0,4.0,150.0,407.0,0.0,2.0,154.0,0.0,4.0,2.0,3.0,7.0


## Data Preparation
<a id='dataprep'></a>
While our dataset comes to us fairly clean, there is still much data preparation to do before we can get to the business of predictive modelling.

First of all, we must integer encode the target feature. It is holds the values (1) for the absence of heart disease, and (2) for the presence of heart disease. Since the presence is our postive feature, we wish to map this to (1) and the absence to (0).

In [22]:
# firstly we integer encode target feature 
print("Target before encoding: ",np.unique(target,return_counts=True))
encoded_target = np.where(target==1,0,1)
print("Target after encoding: ",np.unique(encoded_target,return_counts=True))

Target before encoding:  (array([1, 2]), array([150, 120]))
Target after encoding:  (array([0, 1]), array([150, 120]))


We now turn to our nominal features which have been integer encoded. As integer encoding assumes an ordering, we consider it bad practice to integer encode nominal features, so we will undo this encoding and repace it with a one-hot encoding scheme.

In [27]:
# the variable "chest_pain_type" or "cpt" has the following values:
data['cpt'].value_counts()
# we map each of them to the following categorical levels
cpt_mappings = {1.0 : 'l1', 2.0 : 'l2', 3.0 : 'l3', 4.0 : 'l4'}
data['cpt'] = data['cpt'].replace(cpt_mappings)

# NO MISSING VALUES - but are there outliers?

# integer encode all binary features: sex, fsb, eia

# NOMINAL AND ORDINAL FEATURES ALREADY INTEGER ENCODED - one-hot encode nominal?

# scale features

0      l4
1      l3
2      l2
3      l4
4      l2
5      l4
6      l3
7      l4
8      l4
9      l4
10     l4
11     l4
12     l3
13     l1
14     l4
15     l4
16     l4
17     l4
18     l1
19     l1
20     l4
21     l2
22     l4
23     l4
24     l2
25     l3
26     l4
27     l3
28     l3
29     l3
       ..
240    l3
241    l3
242    l4
243    l4
244    l3
245    l4
246    l4
247    l3
248    l3
249    l4
250    l4
251    l2
252    l4
253    l3
254    l3
255    l2
256    l3
257    l4
258    l3
259    l4
260    l3
261    l4
262    l2
263    l2
264    l2
265    l3
266    l2
267    l2
268    l4
269    l4
Name: cpt, Length: 270, dtype: object

## Data Exploration & Visualisation